### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 300].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Optimization- Attempt 1

In [14]:
# Define the model
# Added hidden node layer, chaged activation to "sigmoid"
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='sigmoid'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='sigmoid'))

#Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 7,601
Trainable params: 7,601
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 59/804 [=>............................] - ETA: 0s - loss: 0.6985 - accuracy: 0.4952  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
153/804 [====>.........................] - ETA: 0s - loss: 0.6911 - accuracy: 0.5262
Epoch 00001: saving model to checkpoints\weights.01.hdf5
248/804 [========>.....................] - ETA: 0s - loss: 0.6813 - accuracy: 0.5548
Epoch 00001: saving model to checkpoints\weights.01.hdf5
348/804 [===========>..................] - ETA: 0s - loss: 0.6702 - accuracy: 0.5786
Epoch 00001: saving model to checkpoints\weights.01.hdf5
432/804 [===============>..............] - ETA: 0s - loss: 0.6618 - accuracy: 0.5943
Epoch 00001: saving model to checkpoints\weights.01.hdf5
532/804 [==================>...........] - ETA: 0s - loss: 0.6536 - accuracy: 0.6088
Epoch 00001: saving model to checkpoints\weights.01.hdf5
694/804 [========================>.....] - ETA: 0s - loss: 0.6431 - accuracy: 0.6263
Epoch 00001: saving model to checkpoints\weights.

182/804 [=====>........................] - ETA: 1s - loss: 0.5400 - accuracy: 0.7345
Epoch 00014: saving model to checkpoints\weights.14.hdf5
281/804 [=========>....................] - ETA: 0s - loss: 0.5419 - accuracy: 0.7348
Epoch 00014: saving model to checkpoints\weights.14.hdf5
414/804 [==============>...............] - ETA: 0s - loss: 0.5432 - accuracy: 0.7349
Epoch 00014: saving model to checkpoints\weights.14.hdf5
520/804 [==================>...........] - ETA: 0s - loss: 0.5446 - accuracy: 0.7344
Epoch 00014: saving model to checkpoints\weights.14.hdf5
626/804 [======================>.......] - ETA: 0s - loss: 0.5455 - accuracy: 0.7341
Epoch 00014: saving model to checkpoints\weights.14.hdf5
727/804 [==========================>...] - ETA: 0s - loss: 0.5460 - accuracy: 0.7339
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7337
Epoch 15/100
  1/804 [..............................] - ETA

329/804 [===========>..................] - ETA: 0s - loss: 0.5359 - accuracy: 0.7469
Epoch 00027: saving model to checkpoints\weights.27.hdf5
459/804 [================>.............] - ETA: 0s - loss: 0.5384 - accuracy: 0.7446
Epoch 00027: saving model to checkpoints\weights.27.hdf5
562/804 [===================>..........] - ETA: 0s - loss: 0.5401 - accuracy: 0.7429
Epoch 00027: saving model to checkpoints\weights.27.hdf5
665/804 [=======================>......] - ETA: 0s - loss: 0.5413 - accuracy: 0.7416
Epoch 00027: saving model to checkpoints\weights.27.hdf5
768/804 [===========================>..] - ETA: 0s - loss: 0.5419 - accuracy: 0.7405
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 1s 979us/step - loss: 0.5421 - accuracy: 0.7403
Epoch 28/100
 67/804 [=>............................] - ETA: 0s - loss: 0.5227 - accuracy: 0.7620
Epoch 00028: saving model to checkpoints\weights.28.hdf5
183/804 [=====>........................] - E

Epoch 00040: saving model to checkpoints\weights.40.hdf5
523/804 [==================>...........] - ETA: 0s - loss: 0.5437 - accuracy: 0.7383
Epoch 00040: saving model to checkpoints\weights.40.hdf5
630/804 [======================>.......] - ETA: 0s - loss: 0.5434 - accuracy: 0.7386
Epoch 00040: saving model to checkpoints\weights.40.hdf5
744/804 [==========================>...] - ETA: 0s - loss: 0.5433 - accuracy: 0.7387
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 1s 984us/step - loss: 0.5432 - accuracy: 0.7386
Epoch 41/100
  1/804 [..............................] - ETA: 0s - loss: 0.4996 - accuracy: 0.8438
Epoch 00041: saving model to checkpoints\weights.41.hdf5
124/804 [===>..........................] - ETA: 0s - loss: 0.5460 - accuracy: 0.7366
Epoch 00041: saving model to checkpoints\weights.41.hdf5
225/804 [=======>......................] - ETA: 0s - loss: 0.5464 - accuracy: 0.7363
Epoch 00041: saving model to checkpoints\wei

560/804 [===================>..........] - ETA: 0s - loss: 0.5469 - accuracy: 0.7330
Epoch 00053: saving model to checkpoints\weights.53.hdf5
668/804 [=======================>......] - ETA: 0s - loss: 0.5460 - accuracy: 0.7336
Epoch 00053: saving model to checkpoints\weights.53.hdf5
774/804 [===========================>..] - ETA: 0s - loss: 0.5453 - accuracy: 0.7341
Epoch 00053: saving model to checkpoints\weights.53.hdf5
804/804 [==============================] - 1s 949us/step - loss: 0.5452 - accuracy: 0.7342
Epoch 54/100
 68/804 [=>............................] - ETA: 0s - loss: 0.5230 - accuracy: 0.7569
Epoch 00054: saving model to checkpoints\weights.54.hdf5
183/804 [=====>........................] - ETA: 0s - loss: 0.5303 - accuracy: 0.7487
Epoch 00054: saving model to checkpoints\weights.54.hdf5
283/804 [=========>....................] - ETA: 0s - loss: 0.5329 - accuracy: 0.7457
Epoch 00054: saving model to checkpoints\weights.54.hdf5
322/804 [===========>..................] - E

Epoch 00066: saving model to checkpoints\weights.66.hdf5
706/804 [=========================>....] - ETA: 0s - loss: 0.5350 - accuracy: 0.7388
Epoch 00066: saving model to checkpoints\weights.66.hdf5
804/804 [==============================] - 1s 958us/step - loss: 0.5354 - accuracy: 0.7388
Epoch 67/100
  1/804 [..............................] - ETA: 0s - loss: 0.5928 - accuracy: 0.6250
Epoch 00067: saving model to checkpoints\weights.67.hdf5
112/804 [===>..........................] - ETA: 0s - loss: 0.5489 - accuracy: 0.7255
Epoch 00067: saving model to checkpoints\weights.67.hdf5
219/804 [=======>......................] - ETA: 0s - loss: 0.5422 - accuracy: 0.7317
Epoch 00067: saving model to checkpoints\weights.67.hdf5
331/804 [===========>..................] - ETA: 0s - loss: 0.5402 - accuracy: 0.7345
Epoch 00067: saving model to checkpoints\weights.67.hdf5
433/804 [===============>..............] - ETA: 0s - loss: 0.5393 - accuracy: 0.7357
Epoch 00067: saving model to checkpoints\wei

744/804 [==========================>...] - ETA: 0s - loss: 0.5363 - accuracy: 0.7423
Epoch 00079: saving model to checkpoints\weights.79.hdf5
804/804 [==============================] - 1s 998us/step - loss: 0.5363 - accuracy: 0.7422
Epoch 80/100
 67/804 [=>............................] - ETA: 0s - loss: 0.5451 - accuracy: 0.7264
Epoch 00080: saving model to checkpoints\weights.80.hdf5
121/804 [===>..........................] - ETA: 0s - loss: 0.5430 - accuracy: 0.7308
Epoch 00080: saving model to checkpoints\weights.80.hdf5
249/804 [========>.....................] - ETA: 0s - loss: 0.5401 - accuracy: 0.7338
Epoch 00080: saving model to checkpoints\weights.80.hdf5
333/804 [===========>..................] - ETA: 0s - loss: 0.5391 - accuracy: 0.7353
Epoch 00080: saving model to checkpoints\weights.80.hdf5
418/804 [==============>...............] - ETA: 0s - loss: 0.5390 - accuracy: 0.7360
Epoch 00080: saving model to checkpoints\weights.80.hdf5
521/804 [==================>...........] - E

448/804 [===============>..............] - ETA: 0s - loss: 0.5288 - accuracy: 0.7450
Epoch 00086: saving model to checkpoints\weights.86.hdf5
551/804 [===================>..........] - ETA: 0s - loss: 0.5296 - accuracy: 0.7445
Epoch 00086: saving model to checkpoints\weights.86.hdf5
657/804 [=======================>......] - ETA: 0s - loss: 0.5301 - accuracy: 0.7442
Epoch 00086: saving model to checkpoints\weights.86.hdf5
699/804 [=========================>....] - ETA: 0s - loss: 0.5304 - accuracy: 0.7441
Epoch 00086: saving model to checkpoints\weights.86.hdf5
804/804 [==============================] - 1s 951us/step - loss: 0.5311 - accuracy: 0.7437
Epoch 87/100
  1/804 [..............................] - ETA: 0s - loss: 0.4522 - accuracy: 0.8438
Epoch 00087: saving model to checkpoints\weights.87.hdf5
124/804 [===>..........................] - ETA: 0s - loss: 0.5386 - accuracy: 0.7408
Epoch 00087: saving model to checkpoints\weights.87.hdf5
230/804 [=======>......................] - E

565/804 [====================>.........] - ETA: 0s - loss: 0.5250 - accuracy: 0.7486
Epoch 00099: saving model to checkpoints\weights.99.hdf5
664/804 [=======================>......] - ETA: 0s - loss: 0.5266 - accuracy: 0.7474
Epoch 00099: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 1s 995us/step - loss: 0.5281 - accuracy: 0.7461
Epoch 100/100
  1/804 [..............................] - ETA: 0s - loss: 0.7288 - accuracy: 0.5625
Epoch 00100: saving model to checkpoints\weights.100.hdf5
 55/804 [=>............................] - ETA: 0s - loss: 0.5528 - accuracy: 0.7284
Epoch 00100: saving model to checkpoints\weights.100.hdf5
172/804 [=====>........................] - ETA: 0s - loss: 0.5433 - accuracy: 0.7350
Epoch 00100: saving model to checkpoints\weights.100.hdf5
275/804 [=========>....................] - ETA: 0s - loss: 0.5400 - accuracy: 0.7375
Epoch 00100: saving model to checkpoints\weights.100.hdf5
362/804 [============>.................

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5573 - accuracy: 0.7257
Loss: 0.5573380589485168, Accuracy: 0.7257142663002014


### Optimization- Attempt 2

In [6]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Dropping 'STATUS' instead of 'NAME'
application_df = application_df.drop(columns=["EIN", "STATUS"])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [8]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()

replace_application = list(application_count[application_count < 300].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()

In [11]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Bin names
name_count = application_df.NAME.value_counts()
replace_name = list(name_count[name_count < 5].index)

# Replace in DataFrame
for name in replace_name:
    application_df.NAME = application_df.NAME.replace(name, "Other")
    
# Check to make sure binning was successful
application_df.NAME.value_counts()

Other                                 19803
PARENT BOOSTER USA INC                 1260
TOPS CLUB INC                           765
UNITED STATES BOWLING CONGRESS INC      700
WASHINGTON STATE UNIVERSITY             492
                                      ...  
RURITAN NATIONAL CORP                     5
SOCIETY OF PLASTICS ENGINEERS             5
NATIONAL HUGUENOT SOCIETY                 5
PTA HAWAII CONGRESS                       5
FAMILY MOTOR COACH ASSOCIATION            5
Name: NAME, Length: 403, dtype: int64

In [13]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

In [15]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

In [16]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               44600     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 49,701
Trainable params: 49,701
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 47/804 [>.............................] - ETA: 0s - loss: 0.6767 - accuracy: 0.6443  
Epoch 00001: saving model to checkpoints\weights.01.hdf5
183/804 [=====>........................] - ETA: 0s - loss: 0.5935 - accuracy: 0.7083
Epoch 00001: saving model to checkpoints\weights.01.hdf5
294/804 [=========>....................] - ETA: 0s - loss: 0.5627 - accuracy: 0.7284
Epoch 00001: saving model to checkpoints\weights.01.hdf5
393/804 [=============>................] - ETA: 0s - loss: 0.5461 - accuracy: 0.7383
Epoch 00001: saving model to checkpoints\weights.01.hdf5
467/804 [================>.............] - ETA: 0s - loss: 0.5372 - accuracy: 0.7434
Epoch 00001: saving model to checkpoints\weights.01.hdf5
591/804 [=====================>........] - ETA: 0s - loss: 0.5252 - accuracy: 0.7497
Epoch 00001: saving model to checkpoints\weights.01.hdf5
688/804 [========================>.....] - ETA: 0s - loss: 0.5178 - accuracy: 0.7536
Epoch 00001: saving model to checkpoints\weights.

132/804 [===>..........................] - ETA: 1s - loss: 0.4024 - accuracy: 0.8123
Epoch 00014: saving model to checkpoints\weights.14.hdf5
248/804 [========>.....................] - ETA: 1s - loss: 0.4030 - accuracy: 0.8116
Epoch 00014: saving model to checkpoints\weights.14.hdf5
347/804 [===========>..................] - ETA: 1s - loss: 0.4041 - accuracy: 0.8104
Epoch 00014: saving model to checkpoints\weights.14.hdf5
445/804 [===============>..............] - ETA: 1s - loss: 0.4053 - accuracy: 0.8095
Epoch 00014: saving model to checkpoints\weights.14.hdf5
545/804 [===================>..........] - ETA: 1s - loss: 0.4061 - accuracy: 0.8089
Epoch 00014: saving model to checkpoints\weights.14.hdf5
634/804 [======================>.......] - ETA: 0s - loss: 0.4068 - accuracy: 0.8084
Epoch 00014: saving model to checkpoints\weights.14.hdf5
744/804 [==========================>...] - ETA: 0s - loss: 0.4076 - accuracy: 0.8077
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/80

292/804 [=========>....................] - ETA: 2s - loss: 0.3909 - accuracy: 0.8192
Epoch 00027: saving model to checkpoints\weights.27.hdf5
394/804 [=============>................] - ETA: 1s - loss: 0.3946 - accuracy: 0.8161
Epoch 00027: saving model to checkpoints\weights.27.hdf5
496/804 [=================>............] - ETA: 1s - loss: 0.3967 - accuracy: 0.8142
Epoch 00027: saving model to checkpoints\weights.27.hdf5
586/804 [====================>.........] - ETA: 0s - loss: 0.3982 - accuracy: 0.8130
Epoch 00027: saving model to checkpoints\weights.27.hdf5
686/804 [========================>.....] - ETA: 0s - loss: 0.3998 - accuracy: 0.8119
Epoch 00027: saving model to checkpoints\weights.27.hdf5
794/804 [============================>.] - ETA: 0s - loss: 0.4011 - accuracy: 0.8110
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 4s 5ms/step - loss: 0.4012 - accuracy: 0.8110
Epoch 28/100
 85/804 [==>...........................] - ETA

443/804 [===============>..............] - ETA: 1s - loss: 0.4032 - accuracy: 0.8090
Epoch 00040: saving model to checkpoints\weights.40.hdf5
544/804 [===================>..........] - ETA: 1s - loss: 0.4042 - accuracy: 0.8084
Epoch 00040: saving model to checkpoints\weights.40.hdf5
636/804 [======================>.......] - ETA: 0s - loss: 0.4045 - accuracy: 0.8083
Epoch 00040: saving model to checkpoints\weights.40.hdf5
719/804 [=========================>....] - ETA: 0s - loss: 0.4046 - accuracy: 0.8083
Epoch 00040: saving model to checkpoints\weights.40.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.4047 - accuracy: 0.8083
Epoch 41/100
  1/804 [..............................] - ETA: 0s - loss: 0.3917 - accuracy: 0.8125
Epoch 00041: saving model to checkpoints\weights.41.hdf5
138/804 [====>.........................] - ETA: 2s - loss: 0.4110 - accuracy: 0.8015
Epoch 00041: saving model to checkpoints\weights.41.hdf5
236/804 [=======>......................] - ETA

 92/804 [==>...........................] - ETA: 1s - loss: 0.3871 - accuracy: 0.8163
Epoch 00047: saving model to checkpoints\weights.47.hdf5
208/804 [======>.......................] - ETA: 1s - loss: 0.3885 - accuracy: 0.8188
Epoch 00047: saving model to checkpoints\weights.47.hdf5
281/804 [=========>....................] - ETA: 1s - loss: 0.3899 - accuracy: 0.8182
Epoch 00047: saving model to checkpoints\weights.47.hdf5
389/804 [=============>................] - ETA: 0s - loss: 0.3921 - accuracy: 0.8168
Epoch 00047: saving model to checkpoints\weights.47.hdf5
496/804 [=================>............] - ETA: 0s - loss: 0.3935 - accuracy: 0.8160
Epoch 00047: saving model to checkpoints\weights.47.hdf5
592/804 [=====================>........] - ETA: 0s - loss: 0.3948 - accuracy: 0.8151
Epoch 00047: saving model to checkpoints\weights.47.hdf5
696/804 [========================>.....] - ETA: 0s - loss: 0.3961 - accuracy: 0.8141
Epoch 00047: saving model to checkpoints\weights.47.hdf5
804/80

147/804 [====>.........................] - ETA: 1s - loss: 0.4043 - accuracy: 0.8035
Epoch 00060: saving model to checkpoints\weights.60.hdf5
263/804 [========>.....................] - ETA: 2s - loss: 0.4049 - accuracy: 0.8032
Epoch 00060: saving model to checkpoints\weights.60.hdf5
361/804 [============>.................] - ETA: 1s - loss: 0.4038 - accuracy: 0.8042
Epoch 00060: saving model to checkpoints\weights.60.hdf5
461/804 [================>.............] - ETA: 1s - loss: 0.4031 - accuracy: 0.8048
Epoch 00060: saving model to checkpoints\weights.60.hdf5
560/804 [===================>..........] - ETA: 1s - loss: 0.4024 - accuracy: 0.8056
Epoch 00060: saving model to checkpoints\weights.60.hdf5
661/804 [=======================>......] - ETA: 0s - loss: 0.4022 - accuracy: 0.8061
Epoch 00060: saving model to checkpoints\weights.60.hdf5
761/804 [===========================>..] - ETA: 0s - loss: 0.4022 - accuracy: 0.8064
Epoch 00060: saving model to checkpoints\weights.60.hdf5
804/80

294/804 [=========>....................] - ETA: 1s - loss: 0.3811 - accuracy: 0.8235
Epoch 00073: saving model to checkpoints\weights.73.hdf5
404/804 [==============>...............] - ETA: 0s - loss: 0.3857 - accuracy: 0.8201
Epoch 00073: saving model to checkpoints\weights.73.hdf5
499/804 [=================>............] - ETA: 0s - loss: 0.3887 - accuracy: 0.8177
Epoch 00073: saving model to checkpoints\weights.73.hdf5
560/804 [===================>..........] - ETA: 0s - loss: 0.3898 - accuracy: 0.8168
Epoch 00073: saving model to checkpoints\weights.73.hdf5
681/804 [========================>.....] - ETA: 0s - loss: 0.3919 - accuracy: 0.8152
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.3934 - accuracy: 0.8141
Epoch 74/100
  1/804 [..............................] - ETA: 0s - loss: 0.2276 - accuracy: 0.9375
Epoch 00074: saving model to checkpoints\weights.74.hdf5
 91/804 [==>...........................] - ETA

344/804 [===========>..................] - ETA: 1s - loss: 0.4004 - accuracy: 0.8032
Epoch 00086: saving model to checkpoints\weights.86.hdf5
460/804 [================>.............] - ETA: 1s - loss: 0.3999 - accuracy: 0.8044
Epoch 00086: saving model to checkpoints\weights.86.hdf5
555/804 [===================>..........] - ETA: 0s - loss: 0.3998 - accuracy: 0.8051
Epoch 00086: saving model to checkpoints\weights.86.hdf5
644/804 [=======================>......] - ETA: 0s - loss: 0.3997 - accuracy: 0.8058
Epoch 00086: saving model to checkpoints\weights.86.hdf5
747/804 [==========================>...] - ETA: 0s - loss: 0.3997 - accuracy: 0.8064
Epoch 00086: saving model to checkpoints\weights.86.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.3996 - accuracy: 0.8067
Epoch 87/100
 50/804 [>.............................] - ETA: 2s - loss: 0.4023 - accuracy: 0.7937
Epoch 00087: saving model to checkpoints\weights.87.hdf5
141/804 [====>.........................] - ETA

506/804 [=================>............] - ETA: 0s - loss: 0.4019 - accuracy: 0.8090
Epoch 00099: saving model to checkpoints\weights.99.hdf5
606/804 [=====================>........] - ETA: 0s - loss: 0.4014 - accuracy: 0.8090
Epoch 00099: saving model to checkpoints\weights.99.hdf5
706/804 [=========================>....] - ETA: 0s - loss: 0.4007 - accuracy: 0.8092
Epoch 00099: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.4002 - accuracy: 0.8093
Epoch 100/100
  1/804 [..............................] - ETA: 3s - loss: 0.2522 - accuracy: 0.9375
Epoch 00100: saving model to checkpoints\weights.100.hdf5
103/804 [==>...........................] - ETA: 3s - loss: 0.3813 - accuracy: 0.8226
Epoch 00100: saving model to checkpoints\weights.100.hdf5
202/804 [======>.......................] - ETA: 3s - loss: 0.3866 - accuracy: 0.8188
Epoch 00100: saving model to checkpoints\weights.100.hdf5
289/804 [=========>....................] -

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4742 - accuracy: 0.7896
Loss: 0.474163681268692, Accuracy: 0.7896209955215454


### Optimization- Attempt 3

In [14]:
# Define the model
# Add two more hidden layers
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 200
hidden_nodes_layer2 = 120
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
# Trying 'tanh' activation
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='tanh'))

# Fourth hidden layer
# Trying 'tanh' activation
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 200)               8800      
_________________________________________________________________
dense_3 (Dense)              (None, 120)               24120     
_________________________________________________________________
dense_4 (Dense)              (None, 60)                7260      
_________________________________________________________________
dense_5 (Dense)              (None, 60)                3660      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 61        
Total params: 43,901
Trainable params: 43,901
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 85/804 [==>...........................] - ETA: 0s - loss: 0.6206 - accuracy: 0.6709
Epoch 00001: saving model to checkpoints\weights.01.hdf5
196/804 [======>.......................] - ETA: 0s - loss: 0.6020 - accuracy: 0.6921
Epoch 00001: saving model to checkpoints\weights.01.hdf5
289/804 [=========>....................] - ETA: 0s - loss: 0.5939 - accuracy: 0.7003
Epoch 00001: saving model to checkpoints\weights.01.hdf5
374/804 [============>.................] - ETA: 0s - loss: 0.5892 - accuracy: 0.7052
Epoch 00001: saving model to checkpoints\weights.01.hdf5
497/804 [=================>............] - ETA: 0s - loss: 0.5848 - accuracy: 0.7094
Epoch 00001: saving model to checkpoints\weights.01.hdf5
598/804 [=====================>........] - ETA: 0s - loss: 0.5823 - accuracy: 0.7114
Epoch 00001: saving model to checkpoints\weights.01.hdf5
699/804 [=========================>....] - ETA: 0s - loss: 0.5804 - accuracy: 0.7128
Epoch 00001: saving model to checkpoints\weights.01

551/804 [===================>..........] - ETA: 0s - loss: 0.5429 - accuracy: 0.7378
Epoch 00007: saving model to checkpoints\weights.07.hdf5
676/804 [========================>.....] - ETA: 0s - loss: 0.5439 - accuracy: 0.7369
Epoch 00007: saving model to checkpoints\weights.07.hdf5
734/804 [==========================>...] - ETA: 0s - loss: 0.5442 - accuracy: 0.7366
Epoch 00007: saving model to checkpoints\weights.07.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7362
Epoch 8/100
 49/804 [>.............................] - ETA: 0s - loss: 0.5464 - accuracy: 0.7240
Epoch 00008: saving model to checkpoints\weights.08.hdf5
165/804 [=====>........................] - ETA: 2s - loss: 0.5453 - accuracy: 0.7286
Epoch 00008: saving model to checkpoints\weights.08.hdf5
270/804 [=========>....................] - ETA: 1s - loss: 0.5474 - accuracy: 0.7279
Epoch 00008: saving model to checkpoints\weights.08.hdf5
355/804 [============>.................] - ETA:

Epoch 00014: saving model to checkpoints\weights.14.hdf5
226/804 [=======>......................] - ETA: 0s - loss: 0.5418 - accuracy: 0.7310
Epoch 00014: saving model to checkpoints\weights.14.hdf5
325/804 [===========>..................] - ETA: 0s - loss: 0.5402 - accuracy: 0.7337
Epoch 00014: saving model to checkpoints\weights.14.hdf5
445/804 [===============>..............] - ETA: 0s - loss: 0.5397 - accuracy: 0.7355
Epoch 00014: saving model to checkpoints\weights.14.hdf5
545/804 [===================>..........] - ETA: 0s - loss: 0.5398 - accuracy: 0.7361
Epoch 00014: saving model to checkpoints\weights.14.hdf5
646/804 [=======================>......] - ETA: 0s - loss: 0.5403 - accuracy: 0.7361
Epoch 00014: saving model to checkpoints\weights.14.hdf5
729/804 [==========================>...] - ETA: 0s - loss: 0.5408 - accuracy: 0.7361
Epoch 00014: saving model to checkpoints\weights.14.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accuracy: 0.7362
Ep

720/804 [=========================>....] - ETA: 0s - loss: 0.5397 - accuracy: 0.7407
Epoch 00020: saving model to checkpoints\weights.20.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 0.7403
Epoch 21/100
 19/804 [..............................] - ETA: 2s - loss: 0.5269 - accuracy: 0.7596
Epoch 00021: saving model to checkpoints\weights.21.hdf5
106/804 [==>...........................] - ETA: 2s - loss: 0.5352 - accuracy: 0.7485
Epoch 00021: saving model to checkpoints\weights.21.hdf5
217/804 [=======>......................] - ETA: 1s - loss: 0.5338 - accuracy: 0.7474
Epoch 00021: saving model to checkpoints\weights.21.hdf5
319/804 [==========>...................] - ETA: 1s - loss: 0.5330 - accuracy: 0.7472
Epoch 00021: saving model to checkpoints\weights.21.hdf5
419/804 [==============>...............] - ETA: 1s - loss: 0.5333 - accuracy: 0.7467
Epoch 00021: saving model to checkpoints\weights.21.hdf5
518/804 [==================>...........] - ETA

394/804 [=============>................] - ETA: 1s - loss: 0.5370 - accuracy: 0.7416
Epoch 00027: saving model to checkpoints\weights.27.hdf5
487/804 [=================>............] - ETA: 0s - loss: 0.5381 - accuracy: 0.7404
Epoch 00027: saving model to checkpoints\weights.27.hdf5
587/804 [====================>.........] - ETA: 0s - loss: 0.5393 - accuracy: 0.7393
Epoch 00027: saving model to checkpoints\weights.27.hdf5
674/804 [========================>.....] - ETA: 0s - loss: 0.5399 - accuracy: 0.7387
Epoch 00027: saving model to checkpoints\weights.27.hdf5
771/804 [===========================>..] - ETA: 0s - loss: 0.5401 - accuracy: 0.7386
Epoch 00027: saving model to checkpoints\weights.27.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5401 - accuracy: 0.7386
Epoch 28/100
 82/804 [==>...........................] - ETA: 1s - loss: 0.5491 - accuracy: 0.7294
Epoch 00028: saving model to checkpoints\weights.28.hdf5
164/804 [=====>........................] - ETA

...

412/804 [==============>...............] - ETA: 1s - loss: 0.5338 - accuracy: 0.7366
Epoch 00073: saving model to checkpoints\weights.73.hdf5
512/804 [==================>...........] - ETA: 1s - loss: 0.5342 - accuracy: 0.7370
Epoch 00073: saving model to checkpoints\weights.73.hdf5
608/804 [=====================>........] - ETA: 0s - loss: 0.5348 - accuracy: 0.7370
Epoch 00073: saving model to checkpoints\weights.73.hdf5
711/804 [=========================>....] - ETA: 0s - loss: 0.5350 - accuracy: 0.7372
Epoch 00073: saving model to checkpoints\weights.73.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5353 - accuracy: 0.7373
Epoch 74/100
  1/804 [..............................] - ETA: 1s - loss: 0.4073 - accuracy: 0.8750
Epoch 00074: saving model to checkpoints\weights.74.hdf5
106/804 [==>...........................] - ETA: 2s - loss: 0.5263 - accuracy: 0.7523
Epoch 00074: saving model to checkpoints\weights.74.hdf5
208/804 [======>.......................] - ETA

804/804 [==============================] - 2s 2ms/step - loss: 0.5286 - accuracy: 0.7447
Epoch 80/100
 79/804 [=>............................] - ETA: 2s - loss: 0.5600 - accuracy: 0.7295
Epoch 00080: saving model to checkpoints\weights.80.hdf5
176/804 [=====>........................] - ETA: 2s - loss: 0.5485 - accuracy: 0.7353
Epoch 00080: saving model to checkpoints\weights.80.hdf5
275/804 [=========>....................] - ETA: 2s - loss: 0.5439 - accuracy: 0.7380
Epoch 00080: saving model to checkpoints\weights.80.hdf5
366/804 [============>.................] - ETA: 1s - loss: 0.5418 - accuracy: 0.7392
Epoch 00080: saving model to checkpoints\weights.80.hdf5
480/804 [================>.............] - ETA: 1s - loss: 0.5405 - accuracy: 0.7398
Epoch 00080: saving model to checkpoints\weights.80.hdf5
576/804 [====================>.........] - ETA: 0s - loss: 0.5401 - accuracy: 0.7398
Epoch 00080: saving model to checkpoints\weights.80.hdf5
678/804 [========================>.....] - ETA

559/804 [===================>..........] - ETA: 0s - loss: 0.5321 - accuracy: 0.7433
Epoch 00086: saving model to checkpoints\weights.86.hdf5
653/804 [=======================>......] - ETA: 0s - loss: 0.5329 - accuracy: 0.7428
Epoch 00086: saving model to checkpoints\weights.86.hdf5
741/804 [==========================>...] - ETA: 0s - loss: 0.5334 - accuracy: 0.7425
Epoch 00086: saving model to checkpoints\weights.86.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5336 - accuracy: 0.7424
Epoch 87/100
 46/804 [>.............................] - ETA: 0s - loss: 0.5503 - accuracy: 0.7260
Epoch 00087: saving model to checkpoints\weights.87.hdf5
150/804 [====>.........................] - ETA: 0s - loss: 0.5482 - accuracy: 0.7276
Epoch 00087: saving model to checkpoints\weights.87.hdf5
254/804 [========>.....................] - ETA: 1s - loss: 0.5453 - accuracy: 0.7306
Epoch 00087: saving model to checkpoints\weights.87.hdf5
341/804 [===========>..................] - ETA

Epoch 00093: saving model to checkpoints\weights.93.hdf5
199/804 [======>.......................] - ETA: 1s - loss: 0.5269 - accuracy: 0.7501
Epoch 00093: saving model to checkpoints\weights.93.hdf5
331/804 [===========>..................] - ETA: 0s - loss: 0.5309 - accuracy: 0.7460
Epoch 00093: saving model to checkpoints\weights.93.hdf5
386/804 [=============>................] - ETA: 0s - loss: 0.5314 - accuracy: 0.7451
Epoch 00093: saving model to checkpoints\weights.93.hdf5
524/804 [==================>...........] - ETA: 0s - loss: 0.5322 - accuracy: 0.7439
Epoch 00093: saving model to checkpoints\weights.93.hdf5
623/804 [======================>.......] - ETA: 0s - loss: 0.5329 - accuracy: 0.7433
Epoch 00093: saving model to checkpoints\weights.93.hdf5
725/804 [==========================>...] - ETA: 0s - loss: 0.5335 - accuracy: 0.7428
Epoch 00093: saving model to checkpoints\weights.93.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5339 - accuracy: 0.7425
Ep

696/804 [========================>.....] - ETA: 0s - loss: 0.5369 - accuracy: 0.7425
Epoch 00099: saving model to checkpoints\weights.99.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5368 - accuracy: 0.7422
Epoch 100/100
  1/804 [..............................] - ETA: 0s - loss: 0.4890 - accuracy: 0.8125
Epoch 00100: saving model to checkpoints\weights.100.hdf5
 84/804 [==>...........................] - ETA: 2s - loss: 0.5161 - accuracy: 0.7606
Epoch 00100: saving model to checkpoints\weights.100.hdf5
176/804 [=====>........................] - ETA: 1s - loss: 0.5209 - accuracy: 0.7567
Epoch 00100: saving model to checkpoints\weights.100.hdf5
298/804 [==========>...................] - ETA: 1s - loss: 0.5240 - accuracy: 0.7538
Epoch 00100: saving model to checkpoints\weights.100.hdf5
391/804 [=============>................] - ETA: 0s - loss: 0.5259 - accuracy: 0.7518
Epoch 00100: saving model to checkpoints\weights.100.hdf5
485/804 [=================>............]

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5568 - accuracy: 0.7259
Loss: 0.5567928552627563, Accuracy: 0.7259474992752075


In [19]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")